In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os, sqlite3

/Users/rpsingh/anaconda3/lib/python3.9/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
currentFolder = os.getcwd()
homeFolder = os.path.dirname(os.path.dirname(os.path.dirname(currentFolder)))
cnxString = f'{homeFolder}/UCB-MFE-python-preprogram/Lectures/Lecture 4/data/data.db'
print (cnxString)

events_db = 'crypto_events_database'

/Users/rpsingh/Documents/MFE_PREPROGRAMS/python/Homeworks/UCB-MFE-python-preprogram/Lectures/Lecture 4/data/data.db


In [3]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

fetching latest date in db

In [4]:
connection = sqlite3.connect(cnxString)
endDate = pd.to_datetime(pd.read_sql(f'''select max(ts) from ohlc''',connection).iloc[0,0])
endDate

Timestamp('2021-12-15 00:00:00')

In [5]:
run_date = last_dow(endDate, 0)
startDate = run_date - timedelta(days=14)
run_date, startDate

(Timestamp('2021-12-13 00:00:00'), Timestamp('2021-11-29 00:00:00'))

checking for complete 2 weeks in the date range

In [6]:
dates = pd.read_sql(f'''select distinct ts from ohlc where ts between \'{startDate.date()}\' and \'{run_date.date()}\'''', connection)
dates['ts'] = pd.to_datetime(dates['ts']).dt.date
dates = dates.drop_duplicates()
print (dates)

             ts
0    2021-11-29
24   2021-11-30
48   2021-12-01
72   2021-12-02
96   2021-12-03
120  2021-12-04
144  2021-12-05
168  2021-12-06
192  2021-12-07
216  2021-12-08
240  2021-12-09
264  2021-12-10
288  2021-12-11
312  2021-12-12


In [7]:
tokens = pd.read_sql(f'''select distinct token from ohlc''', connection)
tokens = tokens.values.transpose().tolist()[0]
pd.DataFrame(tokens,columns=['token'])
print (tokens)

['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV', 'AAVE', 'COMP']


In [8]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [9]:
for token in tokens:
        print(f'running for {token} for date {run_date - timedelta(days=14)} to {run_date}', end='\r')
        res = pm.execute_notebook(
            'hw_4_divya_singh_template.ipynb',
            f'{base_output_folder}/hw_4_divya_singh_{token}_{run_date}.ipynb',
            parameters = {
                'from_date': f'{run_date - timedelta(days=14)}',
                'to_date': f'{run_date}',
                'token': token,
                'connection_string': cnxString,
                'events_db': events_db
            }
        )

Input notebook does not contain a cell with tag 'parameters'


Executing:   0%|          | 0/25 [00:00<?, ?cell/s]

Input notebook does not contain a cell with tag 'parameters'


Executing:   0%|          | 0/25 [00:00<?, ?cell/s]

Input notebook does not contain a cell with tag 'parameters'


...

Executing:   0%|          | 0/25 [00:00<?, ?cell/s]

Input notebook does not contain a cell with tag 'parameters'


Executing:   0%|          | 0/25 [00:00<?, ?cell/s]

Input notebook does not contain a cell with tag 'parameters'


Executing:   0%|          | 0/25 [00:00<?, ?cell/s]

In [10]:
nbs = sb.read_notebooks('outputs')

In [20]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'avg_hourly_return': scrap['prices'].data.hourly_return.mean(),
        'avg_volatility': scrap['prices'].data.hourly_volatility.mean(),
        'max_drawdown': scrap['prices'].data.max_drawdown.min(),
        'spread':scrap['prices'].data.close.iloc[-1] - scrap['prices'].data.close.iloc[0]})
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,avg_hourly_return,avg_volatility,max_drawdown,spread
0,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
1,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
2,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
3,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
4,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
5,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
6,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
7,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
8,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
9,BTC,2021-11-05,2021-11-11,0.000411,0.086268,-0.068122,3715.35
